# Modeling and Evaluation

This notebook trains baseline and tree-based machine learning models to predict
customer churn and evaluates their performance using appropriate classification metrics.


**1. Imports**

In [10]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import (
    roc_auc_score,
    classification_report,
    confusion_matrix
)

from sklearn.impute import SimpleImputer

**2. Load Data & Preprocessing**

In [11]:
df = pd.read_csv("../data/raw_data.csv")
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

X = df.drop("Churn", axis=1)
y = df["Churn"].map({"Yes": 1, "No": 0})

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [12]:
num_features = ["tenure", "MonthlyCharges", "TotalCharges"]
cat_features = [col for col in X.columns if col not in num_features]

## Data Preparation

The same preprocessing pipeline from the feature engineering stage is reused
to ensure consistency and prevent data leakage.

In [13]:
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", categorical_transformer, cat_features)
    ]
)


**3. Baseline Model: Logistic Regression**

## Baseline Model: Logistic Regression

Logistic regression is used as a baseline due to its simplicity and interpretability.

In [14]:
log_reg = Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", LogisticRegression(max_iter=1000))
    ]
)

log_reg.fit(X_train, y_train)

y_pred_lr = log_reg.predict(X_test)
y_prob_lr = log_reg.predict_proba(X_test)[:, 1]

roc_auc_lr = roc_auc_score(y_test, y_prob_lr)
roc_auc_lr

0.8416750626469297

In [15]:
print(classification_report(y_test, y_pred_lr))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87      1035
           1       0.66      0.56      0.61       374

    accuracy                           0.81      1409
   macro avg       0.75      0.73      0.74      1409
weighted avg       0.80      0.81      0.80      1409



### Logistic Regression Results

Logistic Regression was used as a baseline model due to its simplicity and interpretability.

The model achieves a reasonable ROC-AUC score, indicating that it is able to
distinguish between churned and non-churned customers better than random guessing.
However, its performance is limited by the linear nature of the decision boundary.

Despite lower predictive power compared to ensemble models, Logistic Regression
provides a transparent benchmark and serves as a reference point for evaluating
more complex approaches.    

### Model Interpretation

The baseline performance achieved by Logistic Regression indicates that customer
churn is partially driven by linear relationships between customer tenure,
pricing variables, and service-related attributes.

This result supports the hypotheses formulated during exploratory data analysis
and motivates the exploration of more flexible models capable of capturing
non-linear effects and feature interactions.

### Summary

In this stage, a complete and reproducible machine learning pipeline was
constructed and evaluated using Logistic Regression as a baseline model.

The results confirm that the dataset contains meaningful patterns related to
customer churn. While the baseline model provides interpretability and stability,
its limitations highlight the need for more expressive models in subsequent stages.

The next step will focus on enhancing predictive performance and analyzing model
decisions through explainability techniques.